#THY Travel Datathon Preselection Case Study

by Duygu Can, Meriç Pakkan, Şule Kütükde, Neslihan Oflaz


Flying passengers can check-in through the web site, applications, kiosks and counters. In this case study content, the train data including the number of seven-month passenger check-in operations is provided. We want you to estimate the number of channels (column Operation_Count) in the csv file attached.

Mount drive to load the data:

In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Dataset Expolaration

Upload the data by reading provided .csv files from Google Drive (change path if needed). There are 808696 samples in the training and 121921 instances in the test sets. Each has 23 features.

In [3]:
import pandas as pd
train_df = pd.read_csv("/content/drive/My Drive/Travel Datathon/Assessment Data/Assessment Train Data.csv")
result_df = pd.read_csv("/content/drive/My Drive/Travel Datathon/Assessment Data/Assessment Result File.csv")
print(train_df.shape)
print(result_df.shape)
train_df.head()

(808696, 23)
(121921, 23)


,Departure_YMD_LMT,Operation_YMD_LMT,Departure_Airport,Operation_Airport,Terminal_Number,Terminal_Name,Operation_Initials,Operation_Sonic_Code,Operation_Channel,Passenger_Title,Passenger_Gender,Inbound_Departure_Airport,Outbound_Arrival_Airport,SWC_FLY,Cabin_Class,SWC_FQTV_Member,Passenger_Baggage_Count,Passenger_Baggage_Weight,SWC_Staff,SWC_CIP_Passenger,SWC_VIP_Passenger,SWC_Has_Infant,Operation_Count
0,20190326,20190326,KDT,KDT,?,032363,KS,?,QC,MISTER,M,IST,?,1,Y,1,0,0,0,1,0,0,1
1,20180321,20180321,KDT,KDT,?,03F960,KS,?,QC,MISTER,M,?,KDT,1,Y,1,0,0,0,1,0,0,1
2,20190322,20190322,KDT,KDT,?,03AA2P,KS,Y011161,QC,MISTER,M,?,?,1,Y,0,0,0,0,0,0,0,1
3,20180515,20180515,KDT,KDT,?,0353D0,MK,?,TS,MISTER,M,?,?,1,Y,1,0,0,0,1,0,0,2
4,20190420,20190420,KDT,KDT,?,03F902,KS,Y013437,QC,MISTER,M,?,?,1,Y,0,0,0,1,0,0,0,1


Check what data types we have:

In [4]:
train_df.dtypes

Departure_YMD_LMT             int64
Operation_YMD_LMT             int64
Departure_Airport            object
Operation_Airport            object
Terminal_Number              object
Terminal_Name                object
Operation_Initials           object
Operation_Sonic_Code         object
Operation_Channel            object
Passenger_Title              object
Passenger_Gender             object
Inbound_Departure_Airport    object
Outbound_Arrival_Airport     object
SWC_FLY                       int64
Cabin_Class                  object
SWC_FQTV_Member               int64
Passenger_Baggage_Count       int64
Passenger_Baggage_Weight      int64
SWC_Staff                     int64
SWC_CIP_Passenger             int64
SWC_VIP_Passenger             int64
SWC_Has_Infant                int64
Operation_Count               int64
dtype: object

In the *Departure_YMD_LMT* and the *Operation_YMD_LMT* colums, date of departure and date of check is stored so it is convenient to convert them to date time objects.

In [0]:
train_df['Departure_YMD_LMT'] = pd.to_datetime(train_df['Departure_YMD_LMT'], format='%Y%m%d')
train_df['Operation_YMD_LMT'] = pd.to_datetime(train_df['Operation_YMD_LMT'], format='%Y%m%d')
result_df['Departure_YMD_LMT'] = pd.to_datetime(result_df['Departure_YMD_LMT'], format='%Y%m%d')
result_df['Operation_YMD_LMT'] = pd.to_datetime(result_df['Operation_YMD_LMT'], format='%Y%m%d')

In [6]:
train_df.dtypes

Departure_YMD_LMT            datetime64[ns]
Operation_YMD_LMT            datetime64[ns]
Departure_Airport                    object
Operation_Airport                    object
Terminal_Number                      object
Terminal_Name                        object
Operation_Initials                   object
Operation_Sonic_Code                 object
Operation_Channel                    object
Passenger_Title                      object
Passenger_Gender                     object
Inbound_Departure_Airport            object
Outbound_Arrival_Airport             object
SWC_FLY                               int64
Cabin_Class                          object
SWC_FQTV_Member                       int64
Passenger_Baggage_Count               int64
Passenger_Baggage_Weight              int64
SWC_Staff                             int64
SWC_CIP_Passenger                     int64
SWC_VIP_Passenger                     int64
SWC_Has_Infant                        int64
Operation_Count                 

In [7]:
train_df.describe()

,SWC_FLY,SWC_FQTV_Member,Passenger_Baggage_Count,Passenger_Baggage_Weight,SWC_Staff,SWC_CIP_Passenger,SWC_VIP_Passenger,SWC_Has_Infant,Operation_Count
count,808696.000000,808696.000000,808696.000000,808696.000000,808696.000000,808696.000000,808696.000000,808696.000000,808696.000000
mean,0.939140,0.359679,0.655413,8.560313,0.029989,0.148156,0.008257,0.011681,1.534792
std,0.239073,0.479907,0.963677,13.667213,0.170557,0.355255,0.090489,0.107444,2.223746
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,15.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,54.000000,970.000000,1.000000,1.000000,1.000000,1.000000,129.000000


Convert *object* datatype to category when needed.

In [0]:
for col_name in train_df:
    if train_df[col_name].dtype.name == 'object':
        train_df[col_name] = train_df[col_name].astype('category')
        result_df[col_name] = result_df[col_name].astype('category')

In [0]:
train_df.dtypes

Departure_YMD_LMT            datetime64[ns]
Operation_YMD_LMT            datetime64[ns]
Departure_Airport                  category
Operation_Airport                  category
Terminal_Number                    category
Terminal_Name                      category
Operation_Initials                 category
Operation_Sonic_Code               category
Operation_Channel                  category
Passenger_Title                    category
Passenger_Gender                   category
Inbound_Departure_Airport          category
Outbound_Arrival_Airport           category
SWC_FLY                               int64
Cabin_Class                        category
SWC_FQTV_Member                       int64
Passenger_Baggage_Count               int64
Passenger_Baggage_Weight              int64
SWC_Staff                             int64
SWC_CIP_Passenger                     int64
SWC_VIP_Passenger                     int64
SWC_Has_Infant                        int64
Operation_Count                 

See the unique levels of the categorical columns. *Departure Airport* has only one value called "KDT". So, it is not informative and should be dropped.

In [0]:
for col_name in train_df.columns:
    if train_df[col_name].dtype.name == 'category':
        print(col_name, ":", train_df[col_name].unique())

Departure_Airport : [KDT]
Categories (1, object): [KDT]
Operation_Airport : [KDT, IST, ZKG, EST, GZT, ..., HRK, ODS, SOF, KNN, MNL]
Length: 143
Categories (143, object): [KDT, IST, ZKG, EST, ..., ODS, SOF, KNN, MNL]
Terminal_Number : [?, P1, P2]
Categories (3, object): [?, P1, P2]
Terminal_Name : [032363, 03F960, 03AA2P, 0353D0, 03F902, ..., 0B19C9, 03554A, 0302CA, 035316, 03550B]
Length: 2685
Categories (2685, object): [032363, 03F960, 03AA2P, 0353D0, ..., 03554A, 0302CA, 035316, 03550B]
Operation_Initials : [KS, MK, SC, EY, Q7, ..., NB, YK, RW, OA, JF]
Length: 273
Categories (272, object): [KS, MK, SC, EY, ..., YK, RW, OA, JF]
Operation_Sonic_Code : [?, Y011161, Y013437, Y027434, Y013171, ..., M025037, Y019509, Y036439, Y015916, Y039203]
Length: 1614
Categories (1614, object): [?, Y011161, Y013437, Y027434, ..., Y019509, Y036439, Y015916,
                            Y039203]
Operation_Channel : [QC, TS, JW, TW, TY, ..., CC, BD, IR, IA, QR]
Length: 13
Categories (13, object): [QC, TS,

Generate *Operation_Channel_Group* as defined in the pdf file.

In [0]:
dict = {"JW": 'Online',
        "TW": 'Online',
        "TS": 'Mobile',
        "JM": 'Mobile',
        "TY":"Counter",
        "QC":"Counter",
        "SC":"Kiosks",
        "IR":"Other",
        "?":"Other",
        "IA":"Other",
        "BD":"Other",
        "CC":"Other",
        "QR":"Other",
        "QP":"Other",
        "QA":"Other"
        }
train_df['Operation_Channel_Group'] = train_df['Operation_Channel'].map(dict)
train_df['Operation_Channel_Group'].unique()

array(['Counter', 'Mobile', 'Online', 'Kiosks', 'Other'], dtype=object)

Do the same for the test set.

In [0]:
result_df['Operation_Channel_Group'] = result_df['Operation_Channel'].map(dict)
result_df['Operation_Channel_Group'].unique()

array(['Mobile', 'Counter', 'Online', 'Kiosks', 'Other'], dtype=object)

### Missing Value Handling

Column percentage of null values in the training and test sets are printed below. At first glance,there are only null values in the *Operation Initials* column, however some unknown values are encoded as "?" in the datasets.

In [0]:
(train_df.isnull().mean()*100).round(4)

Departure_YMD_LMT            0.0000
Operation_YMD_LMT            0.0000
Departure_Airport            0.0000
Operation_Airport            0.0000
Terminal_Number              0.0000
Terminal_Name                0.0000
Operation_Initials           0.0156
Operation_Sonic_Code         0.0000
Operation_Channel            0.0000
Passenger_Title              0.0000
Passenger_Gender             0.0000
Inbound_Departure_Airport    0.0000
Outbound_Arrival_Airport     0.0000
SWC_FLY                      0.0000
Cabin_Class                  0.0000
SWC_FQTV_Member              0.0000
Passenger_Baggage_Count      0.0000
Passenger_Baggage_Weight     0.0000
SWC_Staff                    0.0000
SWC_CIP_Passenger            0.0000
SWC_VIP_Passenger            0.0000
SWC_Has_Infant               0.0000
Operation_Count              0.0000
Operation_Channel_Group      0.0000
dtype: float64

In [0]:
(result_df.isnull().mean()*100).round(4)

Departure_YMD_LMT            0.0000
Operation_YMD_LMT            0.0000
Departure_Airport            0.0000
Operation_Airport            0.0000
Terminal_Number              0.0000
Terminal_Name                0.0000
Operation_Initials           0.0033
Operation_Sonic_Code         0.0000
Operation_Channel            0.0000
Passenger_Title              0.0000
Passenger_Gender             0.0000
Inbound_Departure_Airport    0.0000
Outbound_Arrival_Airport     0.0000
SWC_FLY                      0.0000
Cabin_Class                  0.0000
SWC_FQTV_Member              0.0000
Passenger_Baggage_Count      0.0000
Passenger_Baggage_Weight     0.0000
SWC_Staff                    0.0000
SWC_CIP_Passenger            0.0000
SWC_VIP_Passenger            0.0000
SWC_Has_Infant               0.0000
Operation_Count              0.0000
Operation_Channel_Group      0.0000
dtype: float64

Column based "?" occurance percentages in the training set:

In [0]:
import numpy as np
def unknown_perc(df):
  print("Column Name\t Percentage")
  for col_name in df.columns:
        if df[col_name].dtype.name == 'category' and (df[col_name] == "?").any():
          count = df[col_name].value_counts(dropna=False)['?']
          percentage = (count/len(df)*100).round(3)
          print(col_name,"\t", percentage)
  return
        
unknown_perc(train_df)

Column Name	 Percentage
Terminal_Number 	 90.618
Operation_Sonic_Code 	 78.927
Operation_Channel 	 2.112
Passenger_Title 	 2.331
Passenger_Gender 	 3.902
Inbound_Departure_Airport 	 91.338
Outbound_Arrival_Airport 	 64.422
Cabin_Class 	 3.409


Column based "?" occurance percentages in the test set:

In [0]:
unknown_perc(result_df)

Column Name	 Percentage
Terminal_Number 	 91.39
Operation_Sonic_Code 	 54.39
Operation_Channel 	 0.871
Passenger_Title 	 2.762
Passenger_Gender 	 0.076
Inbound_Departure_Airport 	 94.52
Outbound_Arrival_Airport 	 65.008


Convert "?" to NA when needed. Apart from *Inbound_Departure_Airport* and *Outbound_Arrival_Airport*, "?" is printed for unknown values (missing). For those specific columns "?" means that there is no inbound or outbound flight (direct flight), so they are encoded as a seperate class called *Unknown*.

In [0]:
import numpy as np
train_df['Terminal_Number'] = train_df['Terminal_Number'].replace('?', np.nan)
train_df['Operation_Channel'] = train_df['Operation_Channel'].replace('?', np.nan)
train_df['Passenger_Title'] = train_df['Passenger_Title'].replace('?', np.nan)
train_df['Passenger_Gender'] = train_df['Passenger_Gender'].replace('?', np.nan)
train_df['Inbound_Departure_Airport'] = train_df['Inbound_Departure_Airport'].replace('?', "Unknown")
train_df['Outbound_Arrival_Airport'] = train_df['Outbound_Arrival_Airport'].replace('?', "Unknown")
train_df['Cabin_Class'] = train_df['Cabin_Class'].replace('?', np.nan)

result_df['Terminal_Number'] = result_df['Terminal_Number'].replace('?', np.nan)
result_df['Operation_Channel'] = result_df['Operation_Channel'].replace('?', np.nan)
result_df['Passenger_Title'] = result_df['Passenger_Title'].replace('?', np.nan)
result_df['Passenger_Gender'] = result_df['Passenger_Gender'].replace('?', np.nan)
result_df['Inbound_Departure_Airport'] = result_df['Inbound_Departure_Airport'].replace('?', "Unknown")
result_df['Outbound_Arrival_Airport'] = result_df['Outbound_Arrival_Airport'].replace('?', "Unknown")
result_df['Cabin_Class'] = result_df['Cabin_Class'].replace('?', np.nan)

Now, the new missing value percentages for the training set becomes:

In [0]:
(train_df.isnull().mean()*100)#.round(4)

Departure_YMD_LMT             0.000000
Operation_YMD_LMT             0.000000
Departure_Airport             0.000000
Operation_Airport             0.000000
Terminal_Number              90.617612
Terminal_Name                 0.000000
Operation_Initials            0.015581
Operation_Sonic_Code          0.000000
Operation_Channel             2.112289
Passenger_Title               2.330666
Passenger_Gender              3.902455
Inbound_Departure_Airport     0.000000
Outbound_Arrival_Airport      0.000000
SWC_FLY                       0.000000
Cabin_Class                   3.409316
SWC_FQTV_Member               0.000000
Passenger_Baggage_Count       0.000000
Passenger_Baggage_Weight      0.000000
SWC_Staff                     0.000000
SWC_CIP_Passenger             0.000000
SWC_VIP_Passenger             0.000000
SWC_Has_Infant                0.000000
Operation_Count               0.000000
Operation_Channel_Group       0.000000
dtype: float64

and for the test set the result is:

In [0]:
(result_df.isnull().mean()*100)#.round(4)

Departure_YMD_LMT             0.000000
Operation_YMD_LMT             0.000000
Departure_Airport             0.000000
Operation_Airport             0.000000
Terminal_Number              91.390327
Terminal_Name                 0.000000
Operation_Initials            0.003281
Operation_Sonic_Code          0.000000
Operation_Channel             0.871056
Passenger_Title               2.761624
Passenger_Gender              0.076279
Inbound_Departure_Airport     0.000000
Outbound_Arrival_Airport      0.000000
SWC_FLY                       0.000000
Cabin_Class                   0.000000
SWC_FQTV_Member               0.000000
Passenger_Baggage_Count       0.000000
Passenger_Baggage_Weight      0.000000
SWC_Staff                     0.000000
SWC_CIP_Passenger             0.000000
SWC_VIP_Passenger             0.000000
SWC_Has_Infant                0.000000
Operation_Count               0.000000
Operation_Channel_Group       0.000000
dtype: float64

Notice that *Terminal_Number* has a missing value ratio over 90%. With a ratio this high, we cannot impute missing values correctly. This column should be dropped, as well as *Departure_Airport* column.


In [0]:
train_df2 = train_df.copy()
result_df2 = result_df.copy()
train_df2 = train_df2.drop(columns = ["Terminal_Number", "Departure_Airport"])
result_df2 = result_df2.drop(columns = ["Terminal_Number", "Departure_Airport"])

Ahad date and time


In [8]:
pip install python-dateutil


In [0]:
from dateutil import parser

In [0]:
train_df3 = train_df.copy()
result_df3 = result_df.copy()

In [0]:
train_df3.insert(3,'weekday','mon')
train_df3['weekday'] = df['date_given'].dt.weekday_name
print(df)

In [39]:
train_df3.insert(5,'Month Operation','mon')
#train_df3 = train_df3.drop(columns = ['weekday1'])
train_df3.head()

,Departure_YMD_LMT,weekday Departure,Month Departure,Operation_YMD_LMT,weekday Operation,Month Operation,Departure_Airport,Operation_Airport,Terminal_Number,Terminal_Name,Operation_Initials,Operation_Sonic_Code,Operation_Channel,Passenger_Title,Passenger_Gender,Inbound_Departure_Airport,Outbound_Arrival_Airport,SWC_FLY,Cabin_Class,SWC_FQTV_Member,Passenger_Baggage_Count,Passenger_Baggage_Weight,SWC_Staff,SWC_CIP_Passenger,SWC_VIP_Passenger,SWC_Has_Infant,Operation_Count
0,2019-03-26,Tuesday,3,2019-03-26,Tuesday,mon,KDT,KDT,?,032363,KS,?,QC,MISTER,M,IST,?,1,Y,1,0,0,0,1,0,0,1
1,2018-03-21,Wednesday,3,2018-03-21,Wednesday,mon,KDT,KDT,?,03F960,KS,?,QC,MISTER,M,?,KDT,1,Y,1,0,0,0,1,0,0,1
2,2019-03-22,Friday,3,2019-03-22,Friday,mon,KDT,KDT,?,03AA2P,KS,Y011161,QC,MISTER,M,?,?,1,Y,0,0,0,0,0,0,0,1
3,2018-05-15,Tuesday,5,2018-05-15,Tuesday,mon,KDT,KDT,?,0353D0,MK,?,TS,MISTER,M,?,?,1,Y,1,0,0,0,1,0,0,2
4,2019-04-20,Saturday,4,2019-04-20,Saturday,mon,KDT,KDT,?,03F902,KS,Y013437,QC,MISTER,M,?,?,1,Y,0,0,0,1,0,0,0,1


In [0]:
train_df3['weekday Departure'] = train_df3['Departure_YMD_LMT'].dt.weekday_name
train_df3['weekday Operation'] = train_df3['Departure_YMD_LMT'].dt.weekday_name

In [40]:
#train_df3['Month Operation'] = pd.DatetimeIndex(train_df3['Operation_YMD_LMT']).month
train_df3.head()

,Departure_YMD_LMT,weekday Departure,Month Departure,Operation_YMD_LMT,weekday Operation,Month Operation,Departure_Airport,Operation_Airport,Terminal_Number,Terminal_Name,Operation_Initials,Operation_Sonic_Code,Operation_Channel,Passenger_Title,Passenger_Gender,Inbound_Departure_Airport,Outbound_Arrival_Airport,SWC_FLY,Cabin_Class,SWC_FQTV_Member,Passenger_Baggage_Count,Passenger_Baggage_Weight,SWC_Staff,SWC_CIP_Passenger,SWC_VIP_Passenger,SWC_Has_Infant,Operation_Count
0,2019-03-26,Tuesday,3,2019-03-26,Tuesday,3,KDT,KDT,?,032363,KS,?,QC,MISTER,M,IST,?,1,Y,1,0,0,0,1,0,0,1
1,2018-03-21,Wednesday,3,2018-03-21,Wednesday,3,KDT,KDT,?,03F960,KS,?,QC,MISTER,M,?,KDT,1,Y,1,0,0,0,1,0,0,1
2,2019-03-22,Friday,3,2019-03-22,Friday,3,KDT,KDT,?,03AA2P,KS,Y011161,QC,MISTER,M,?,?,1,Y,0,0,0,0,0,0,0,1
3,2018-05-15,Tuesday,5,2018-05-15,Tuesday,5,KDT,KDT,?,0353D0,MK,?,TS,MISTER,M,?,?,1,Y,1,0,0,0,1,0,0,2
4,2019-04-20,Saturday,4,2019-04-20,Saturday,4,KDT,KDT,?,03F902,KS,Y013437,QC,MISTER,M,?,?,1,Y,0,0,0,1,0,0,0,1
